## Data Collection

Main data collection is processed by running ['update.py'](https://github.com/cjunwon/Youtube-Data-Analysis/blob/main/update.py) on a Flask ['server.py'](https://github.com/cjunwon/Youtube-Data-Analysis/blob/main/server.py) scheduled through ngrok and Invictify.

This collection pipeline automatically updates the AWS MySQL RDS database with up-to-date channel statistics depending on the configured Invictify schedule.

In [1]:
from MySQL_DB_connect_functions import *
from MySQL_DB_update_functions import *
from youtube_api_functions import *

In [2]:
channel_id_list = ['UCIRYBXDze5krPDzAEOxFGVA'] #TheGuardian,NYTimes(UCqnbDFdCpuN8CMEg0VuEBqA)

In [3]:
# using youtube_api_functions.py:
youtube_obj = build_yt_API_object() # builds Youtube API object
video_df = create_video_df(youtube_obj, channel_id_list) # store API data into pandas df
processed_video_df = clean_video_df(video_df) # run df through cleaning function

In [4]:
processed_video_df

,video_id,channelTitle,title,description,publishedAt,viewCount,likeCount,favoriteCount,commentCount,caption,publishDayName,durationSecs,tagCount
0,ZUdFzjIE8f0,Guardian News,Typhoon Hinnamnor: two rescued from undergroun...,Two people in South Korea were rescued in a dr...,2022-09-07,453.0,16.0,0.0,2.0,0,Wednesday,86.0,34
1,1ZCTQwljRqs,Guardian News,Liz Truss faces her first prime minister's que...,Liz Truss answers questions during her first P...,2022-09-07,27438.0,562.0,0.0,63.0,0,Wednesday,3741.0,36
2,rJngREUNLR8,Guardian News,Putin threatens to ‘freeze’ west by cutting ga...,Vladimir Putin has threatened to cut off energ...,2022-09-07,4638.0,101.0,0.0,109.0,0,Wednesday,116.0,28
3,fkj2ZNJxN_8,Guardian News,Putin says election of Truss as UK leader ‘far...,Vladimir Putin has said the way Britain choose...,2022-09-07,5621.0,179.0,0.0,143.0,0,Wednesday,51.0,32
4,DMaVscjesfE,Guardian News,Thérèse Coffey's alarm goes off during LBC int...,Thérèse Coffey's alarm went off during an inte...,2022-09-07,3986.0,85.0,0.0,46.0,0,Wednesday,27.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2BYy3rEWF-8,Guardian News,Chinese foreign minister on Pelosi's Taiwan vi...,Chinese foreign minister Wang Yi has described...,2022-08-04,164994.0,1530.0,0.0,1901.0,0,Thursday,55.0,35
196,2Lmwy0NrBww,Guardian News,Volcano erupts near Iceland's capital in seism...,"After an eight-month break, eruptions have res...",2022-08-04,34797.0,409.0,0.0,47.0,0,Thursday,59.0,34
197,oKKw2FPiNus,Guardian News,Liz Truss: regional pay boards not part of cen...,Liz Truss has said regional pay boards were no...,2022-08-03,44033.0,102.0,0.0,199.0,0,Wednesday,111.0,31
198,SNSPxmsrdJI,Guardian News,62-year-old French sailor saved by Spanish coa...,A French sailor was saved by the Spanish coast...,2022-08-03,39289.0,121.0,0.0,27.0,0,Wednesday,54.0,38
